In [1]:
# Copyright (c) 2016-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree. An additional grant
# of patent rights can be found in the PATENTS file in the same directory.


import numpy as np
import time
from scipy.sparse import csr_matrix
from sklearn.datasets import fetch_20newsgroups
from sklearn.neighbors import LSHForest
from sklearn.feature_extraction import DictVectorizer

In [2]:
# make sure you run 'python setup.py install' first!
import pysparnn

# Fetch data

In [3]:
dataset = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)

docs = np.array([x.split() for x in dataset.data])
datas = np.array(range(len(docs)))

# Build LSH & PySparNN indexes

In [4]:
class SNNSearch:
    def __init__(self, docs, datas):
        
        self.dv = DictVectorizer()
        dicts = []
        for d in docs:
            dicts.append(dict([(w, 1) for w in d]))
        self.dv.fit(dicts)
        features = csr_matrix(self.dv.transform(dicts), dtype=int)
        self.cp = pysparnn.ClusterIndex(features, datas, 
                                        pysparnn.matrix_distance.UnitCosineDistance)
        
    def search(self, docs):
        dicts = []
        for d in docs:
            dicts.append(dict([(w, 1) for w in d]))
        features = csr_matrix(self.dv.transform(dicts), dtype=int)
        return self.cp.search(features, return_metric=False, k=1, k_clusters=1, max_distance=0.05)
        

t0 = time.time()
snn_search = SNNSearch(docs, datas)
print(time.time() - t0)

5.12235403061


In [5]:
class LSHSearch:
    def __init__(self, docs):
        self.lshf = LSHForest(n_estimators=1, n_candidates=1,
                     n_neighbors=1)
        self.dv = DictVectorizer()
        dicts = []
        for d in docs:
            dicts.append(dict([(w, 1) for w in d]))
        self.dv.fit(dicts)
        features = self.dv.transform(dicts)
        # floats are faster
        # features = csr_matrix(features, dtype=int)
        self.lshf.fit(features)
        
    def search(self, docs):
        dicts = []
        for d in docs:
            dicts.append(dict([(w, 1) for w in d]))
        features = self.dv.transform(dicts)
        # floats are faster
        # features = csr_matrix(features, dtype=int)
        return self.lshf.kneighbors(features, return_distance=False)
    
t0 = time.time()    
lsh = LSHSearch(docs) 
print(time.time() - t0)


4.1826338768


In [ ]:
# code that will measure query time and accuracy
from pysparnn_utils import time_it_n

In [6]:
lsh_time, lsh_accuracy = time_it_n(lsh, docs)
print('LSH median time per query: {0}'.format(lsh_time)) 
print('LSH median accuracy: {0}'.format(lsh_accuracy)) 

LSH median time per query: 0.00371722579002
LSH median accuracy: 1.0


In [7]:
snn_time, snn_accuracy = time_it_n(snn_search, docs)
print('PySparNN median time per query: {0}'.format(snn_time)) 
print('PySparNN median accuracy: {0}'.format(snn_accuracy)) 

PySparNN median time per query: 0.00212025046349
PySparNN median accuracy: 1.0


In [8]:
lsh_time / snn_time

1.7532012627945091

In [9]:
(snn_time - lsh_time) / lsh_time

-0.42961483018438312